# History tables to mariadb

## peek

### imports

In [2]:
import os, pandas as pd, numpy as np

In [3]:
datapath = '../data/raw/jawiki/dumps_unzipped/'

### head

In [17]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).shape

(10, 70)

In [19]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,0:10]

,0,1,2,3,4,5,6,7,8,9
0,jawiki,revision,create,2013-01-01 00:00:10.0,/* 関連項目 */,NaN,219.101.94.150,219.101.94.150,NaN,NaN
1,jawiki,revision,create,2013-01-01 00:00:17.0,/* 中継局概要 */,NaN,219.101.94.150,219.101.94.150,NaN,NaN
2,jawiki,revision,create,2013-01-01 00:00:21.0,NaN,NaN,219.101.94.150,219.101.94.150,NaN,NaN
3,jawiki,revision,create,2013-01-01 00:00:25.0,/* 関連項目 */,NaN,219.101.94.150,219.101.94.150,NaN,NaN
4,jawiki,revision,create,2013-01-01 00:00:25.0,カテゴリの一部変更,12357.0,Yamato-i,Yamato-i,NaN,NaN
5,jawiki,revision,create,2013-01-01 00:00:29.0,NaN,600690.0,Yamayamada,Yamayamada,NaN,NaN
6,jawiki,revision,create,2013-01-01 00:00:32.0,/* 放送エリアなど */,NaN,219.101.94.150,219.101.94.150,NaN,NaN
7,jawiki,revision,create,2013-01-01 00:00:36.0,NaN,NaN,219.101.94.150,219.101.94.150,NaN,NaN
8,jawiki,revision,create,2013-01-01 00:00:48.0,r2.7.3) (ロボットによる: [[gl:Clostridium]]を追加,59151.0,Escarbot,Escarbot,NaN,NaN
9,jawiki,revision,create,2013-01-01 00:00:54.0,/* 概要 */,NaN,116.80.228.164,116.80.228.164,NaN,NaN


In [20]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,10:20]

,10,11,12,13,14,15,16,17,18,19
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
4,NaN,extendedconfirmed,NaN,NaN,False,False,False,False,NaN,NaN
5,NaN,NaN,NaN,NaN,True,False,False,False,2012-02-26 11:45:47.0,2012-02-26 11:45:49.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
8,bot,NaN,"name,group",name,True,False,False,False,2006-06-04 18:00:16.0,2006-06-04 18:00:16.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN


In [21]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,20:30]

,20,21,22,23,24,25,26,27,28,29
0,NaN,NaN,NaN,2414774,大間材木テレビ中継局,大間材木テレビ中継局,0,True,0,True
1,NaN,NaN,NaN,1185684,片貝中継局,片貝テレビ中継局,0,True,0,True
2,NaN,NaN,NaN,1202040,NHK三陸テレビ中継局,NHK三陸テレビ中継局,0,True,0,True
3,NaN,NaN,NaN,2078520,岩泉惣畑テレビ中継局,岩泉惣畑テレビ中継局,0,True,0,True
4,2004-10-09 14:57:01.0,25247.0,49.0,1414804,国鉄ホキ9500形貨車,国鉄ホキ9500形貨車,0,True,0,True
5,2012-02-26 11:47:42.0,529.0,225.0,1613405,UROBOROS,UROBOROS,0,True,0,True
6,NaN,NaN,NaN,1283096,NHK胆沢市野々テレビ中継局,NHK胆沢市野々テレビ中継局,0,True,0,True
7,NaN,NaN,NaN,1003300,辺土名テレビ中継局,辺土名テレビ中継局,0,True,0,True
8,2006-06-04 18:01:44.0,175107.0,625.0,556620,クロストリジウム属,クロストリジウム属,0,True,0,True
9,NaN,NaN,NaN,209218,お笑いオンステージ,お笑いオンステージ,0,True,0,True


In [22]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,30:40]

,30,31,32,33,34,35,36,37,38,39
0,True,False,2011-08-23 15:00:28.0,2011-08-23 15:00:28.0,12,6771598,NaN,NaN,NaN,NaN
1,False,False,2007-10-08 03:33:18.0,2007-10-08 03:33:18.0,32,8974810,NaN,NaN,NaN,NaN
2,False,False,2007-10-21 01:19:51.0,2007-10-21 01:19:51.0,14,23793706,NaN,NaN,NaN,NaN
3,False,False,2010-05-02 14:04:34.0,2010-05-02 14:04:34.0,11,389,NaN,NaN,NaN,NaN
4,False,False,2008-04-14 11:48:50.0,2008-04-14 11:48:50.0,44,341692,NaN,NaN,NaN,NaN
5,False,False,2008-11-12 13:42:32.0,2008-11-12 13:42:32.0,109,499153,NaN,NaN,NaN,NaN
6,False,False,2007-12-27 01:13:10.0,2007-12-27 01:13:10.0,27,23794630,NaN,NaN,NaN,NaN
7,False,False,2007-05-10 17:20:22.0,2007-05-10 17:20:22.0,32,2775240,NaN,NaN,NaN,NaN
8,False,False,2006-06-01 13:04:01.0,2006-06-01 13:04:01.0,65,13108024,NaN,NaN,NaN,NaN
9,False,False,2005-02-17 05:20:48.0,2005-02-17 05:20:48.0,74,3533755,NaN,NaN,NaN,NaN


In [23]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,40:50]

,40,41,42,43,44,45,46,47,48,49
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,50:60]

,50,51,52,53,54,55,56,57,58,59
0,NaN,NaN,45611822,44596655,False,NaN,False,3283,-1,h5njsxb8189dwoab2z5uh3wvsvn3dh0
1,NaN,NaN,45611823,44232137,False,NaN,False,3877,-16,hwp0qzjrb7a9qp46qbcii8qnk9aie2e
2,NaN,NaN,45611824,41882385,False,NaN,False,2325,12,rc01e5nzqdiqret0snpjqeqfwghpb9l
3,NaN,NaN,45611825,45611800,False,NaN,False,2147,1,hm3lj6sslfo2uy9ylvklty2i1pqnzyw
4,NaN,NaN,45611826,45554142,True,NaN,False,5080,-6,row3mb36wjvy2towqpqz2dlisf5y2fz
5,NaN,NaN,45611827,45530069,False,NaN,False,17981,514,47xwa7704n0c10kxhn1tumi8ij28hsh
6,NaN,NaN,45611828,41882141,False,NaN,False,2287,9,qmvlgunfihkab8lwzu8l54dzuipw5fd
7,NaN,NaN,45611829,45193367,False,NaN,False,4585,55,2b3y3yvolpchqb02yhfprv9yehnefbb
8,NaN,NaN,45611830,43584236,True,NaN,False,7343,19,4whh2nz4l8a86yuk2dz5infy26xwuhi
9,NaN,NaN,45611831,45082986,False,NaN,False,9577,115,9tejs7u2eyhegf6bxpsz4e3k339y5nb


In [25]:
pd.read_csv(datapath+'2021-12.jawiki.2013.tsv', sep='\t', nrows=10, header=None).iloc[:,60:70]

,60,61,62,63,64,65,66,67,68,69
0,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
1,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
2,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
3,NaN,NaN,False,NaN,False,NaN,NaN,True,False,NaN
4,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
5,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
6,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
7,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
8,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN
9,NaN,NaN,False,NaN,False,NaN,NaN,False,False,NaN


## go

```bash
mysql --user=root --password=XXXXXXXX
```

```sql
CREATE DATABASE ja_hist CHARACTER SET utf8 COLLATE utf8_bin;
USE jawiki;
GRANT ALL PRIVILEGES ON jawiki TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
GRANT ALL PRIVILEGES ON jawiki.* TO bhrdwj@localhost IDENTIFIED BY XXXXXXX;
EXIT
```